In [7]:
!pip install pandas
!pip install selenium


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [8]:
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

In [9]:
# LIST_LINK_PRODUCT = []

# for i in range(11,100):
#     try:
#         response = requests.get("https://alonhadat.com.vn/can-ban-nha-ha-noi-t1/trang-" + str(i + 1) + ".htm")
#         soup = BeautifulSoup(response.content, "html.parser")
#         list_link = soup.findAll('div', 'ct_title')          
#         for item in list_link:
#             link = item.find('a', 'vip').get('href')
#             LIST_LINK_PRODUCT.append(
#                 {
#                     'link': 'https://alonhadat.com.vn' + link
#                 }
#             )
#         print('####Done page' + str(i + 1))
#     except Exception as e:
#         print('####Error get link ' + str(i + 1))
#         print(e)

In [10]:
# print(LIST_LINK_PRODUCT)

In [11]:
# # Set up Selenium WebDriver (you need to have appropriate drivers installed)
# driver = webdriver.Chrome()  # You can use other browsers and corresponding drivers too

# # Load the page
# driver.get("https://bds.com.vn/mua-ban-nha-dat-ha-noi-page2000")

# # Wait for the page to load completely (you might need to adjust the time)
# driver.implicitly_wait(10)

# # Get the page source after JavaScript has rendered the content
# page_source = driver.page_source

# # Now you can use BeautifulSoup to parse the page source
# soup = BeautifulSoup(page_source, "html.parser")

# # Find the link
# anchor_tags = soup.findAll('a', 'js__product-link-for-product-id')
# anchor_tag = anchor_tags[0]

# # Extract the href attribute
# if anchor_tag:
#     href = anchor_tag.get('href')
#     print(href)
# else:
#     print("No anchor tag found.")

# # Close the browser
# driver.quit()

In [13]:
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def fetch_page_source(url):
    try:
        with webdriver.Chrome() as driver:
            driver.get(url)
            return driver.page_source
    except Exception as e:
        logging.error(f"Error fetching page source for URL: {url}")
        logging.exception(e)
        return None

def extract_links(page_source):
    try:
        soup = BeautifulSoup(page_source, "html.parser")
        links = soup.findAll('a', 'js__product-link-for-product-id')
        return [{'link': f"https://batdongsan.com.vn{link.get('href')}"} for link in links]
    except Exception as e:
        logging.error("Error extracting links from page source")
        logging.exception(e)
        return []

def crawl_page(url):
    logging.info(f"Crawling URL: {url}")
    page_source = fetch_page_source(url)
    if not page_source:
        logging.error(f"No page source found for URL: {url}")
        return []

    links = extract_links(page_source)
    if not links:
        logging.error(f"No links found for URL: {url}")
    return links

def crawl_links(base_url, page_range):
    all_links = []
    urls = [f"{base_url}/p{i}" for i in page_range]

    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_url = {executor.submit(crawl_page, url): url for url in urls}
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                links = future.result()
                all_links.extend(links)
                logging.info(f"Done processing URL: {url}")
            except Exception as e:
                logging.error(f"Error processing URL: {url}")
                logging.exception(e)

    return all_links

def save_links_to_csv(links, filename):
    df = pd.DataFrame(links)
    df.to_csv(filename, index=False)

In [21]:
dt = crawl_links('https://batdongsan.com.vn/ban-nha-dat-ha-noi', range(1, 1000))

2024-05-15 17:07:28,843 - INFO - Crawling URL: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p1


2024-05-15 17:07:28,844 - INFO - Crawling URL: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p2
2024-05-15 17:07:28,844 - INFO - Crawling URL: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p3
2024-05-15 17:07:28,844 - INFO - Crawling URL: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p4
2024-05-15 17:07:28,846 - INFO - Crawling URL: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p5
2024-05-15 17:07:35,758 - INFO - Crawling URL: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p6
2024-05-15 17:07:35,766 - INFO - Done processing URL: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p1
2024-05-15 17:07:36,267 - INFO - Crawling URL: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p7
2024-05-15 17:07:36,268 - INFO - Done processing URL: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p4
2024-05-15 17:07:40,226 - INFO - Crawling URL: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p8
2024-05-15 17:07:40,226 - INFO - Done processing URL: https://batdongsan.com.vn/ban-nha-dat-ha-noi/p2
2024-05-15 17:07:40,957 - INF

In [23]:
save_links_to_csv(dt, 'data/links/link_data.csv')

In [9]:
# page_range = range(10)
# LINK_PRIVATE_HOUSE = 'https://batdongsan.com.vn/ban-nha-rieng-ha-noi'
# LIST_LINK_PRIVATE_HOUSE = crawl_links(LINK_PRIVATE_HOUSE, page_range)

# LINK_VILLA_HOUSE = 'https://batdongsan.com.vn/ban-nha-biet-thu-lien-ke-ha-noi'
# LIST_LINK_VILLA_HOUSE = crawl_links(LINK_VILLA_HOUSE, page_range)

# LINK_TOWN_HOUSE = 'https://batdongsan.com.vn/ban-nha-mat-pho-ha-noi'
# LIST_LINK_TOWN_HOUSE = crawl_links(LINK_TOWN_HOUSE, page_range)

# LINK_SHOP_COM_HOUSE = 'https://batdongsan.com.vn/ban-shophouse-nha-pho-thuong-mai-ha-noi'
# LIST_LINK_SHOP_COM_HOUSE = crawl_links(LINK_TOWN_HOUSE, page_range)

# df = pd.DataFrame(LINK_PRIVATE_HOUSE)
# df.to_csv('data/links/link_private_house_data.csv')

# df = pd.DataFrame(LINK_VILLA_HOUSE)
# df.to_csv('data/links/link_villa_house_data.csv')

# df = pd.DataFrame(LINK_TOWN_HOUSE)
# df.to_csv('data/links/link_town_house_data.csv')

# df = pd.DataFrame(LINK_SHOP_COM_HOUSE)
# df.to_csv('data/links/link_shop_com_house_data.csv')

# df = pd.DataFrame(LIST_LINK_PRODUCT)
# existing_df = pd.read_csv('data/link_data.csv')
# last_index = existing_df.index.max()
# df.index = range(last_index + 1, last_index + 1 + len(df))
# df.to_csv('data/link_data.csv', mode='a', header=False, index=True)
page_range = range(100)

urls_and_files = [
    ('https://batdongsan.com.vn/ban-nha-rieng-ha-noi', 'data/links/link_private_house_data.csv'),
    # ('https://batdongsan.com.vn/ban-nha-biet-thu-lien-ke-ha-noi', 'data/links/link_villa_house_data.csv'),
    # ('https://batdongsan.com.vn/ban-nha-mat-pho-ha-noi', 'data/links/link_town_house_data.csv'),
    # ('https://batdongsan.com.vn/ban-shophouse-nha-pho-thuong-mai-ha-noi', 'data/links/link_shop_com_house_data.csv')
]

with ThreadPoolExecutor(max_workers=4) as executor:
    futures = {executor.submit(crawl_links, url, page_range): filename for url, filename in urls_and_files}
    for future in as_completed(futures):
        filename = futures[future]
        try:
            links = future.result()
            save_links_to_csv(links, filename)
            logging.info(f"Saved links to {filename}")
        except Exception as e:
            logging.error(f"Error saving links to {filename}")
            logging.exception(e)

2024-05-15 15:39:08,517 - INFO - Crawling URL: https://alonhadat.com.vn/can-ban-nha-ha-noi-t1/trang-0.htm
2024-05-15 15:39:08,520 - INFO - Crawling URL: https://alonhadat.com.vn/can-ban-nha-ha-noi-t1/trang-1.htm
2024-05-15 15:39:08,520 - INFO - Crawling URL: https://alonhadat.com.vn/can-ban-nha-ha-noi-t1/trang-2.htm
2024-05-15 15:39:11,936 - ERROR - No links found for URL: https://alonhadat.com.vn/can-ban-nha-ha-noi-t1/trang-2.htm
2024-05-15 15:39:11,944 - ERROR - No links found for URL: https://alonhadat.com.vn/can-ban-nha-ha-noi-t1/trang-1.htm
2024-05-15 15:39:11,946 - INFO - Done processing URL: https://alonhadat.com.vn/can-ban-nha-ha-noi-t1/trang-2.htm
2024-05-15 15:39:11,950 - INFO - Done processing URL: https://alonhadat.com.vn/can-ban-nha-ha-noi-t1/trang-1.htm
2024-05-15 15:39:12,032 - ERROR - No links found for URL: https://alonhadat.com.vn/can-ban-nha-ha-noi-t1/trang-0.htm
2024-05-15 15:39:12,033 - INFO - Done processing URL: https://alonhadat.com.vn/can-ban-nha-ha-noi-t1/tran

In [5]:
# Set up Selenium WebDriver
driver = webdriver.Chrome() 

url = "https://batdongsan.com.vn/ban-nha-mat-pho-duong-nguyen-duc-canh-phuong-hoang-van-thu-4/ban-co-via-he-dien-tich-220-8-m2-gia-158-tr-m2-pr39832145"
driver.get(url)

# Initialize a dictionary to store property details with default values
property_details = {
    'ward': '',
    'district': '',
    'price_per_sqm': '',
    'posted_date': '',
    'area': '',
    'price': '',
    'legal_document': '',
    'interior': '',
    'num_bedrooms': '',
    'num_bathrooms': '',
    'num_floors': '',
    'house_orientation': '',
    'balcony_orientation': '',
    'entrance': '',
    'frontage': ''
}

# Set up Selenium WebDriver
with webdriver.Chrome() as driver:
    driver.get(url)
    
    # Get the page source
    page_source = driver.page_source

soup = BeautifulSoup(page_source, "html.parser")
# Find the breadcrumb element
breadcrumb = soup.find('div', class_='re__breadcrumb')

if breadcrumb is None:
    raise ValueError("breadcrumb not found")

level_4_link = breadcrumb.find('a', {'level': '4'})
print(level_4_link.text)
# with open('data/test.html', 'w') as file:
#     file.write(soup.prettify())

# # Split the address into parts
# address_parts = soup.find('span', class_='re__pr-short-description').text.split(', ')


# # Extract ward and district
# ward = address_parts[-3]
# property_details['ward'] = ward
# district = address_parts[-2]
# property_details['district'] = district

# # Find the span containing the price per square meter
# price_per_sqm_span = soup.find('span', class_='ext')

# # Extract the text from the span
# price_per_sqm = price_per_sqm_span.text.strip()

# property_details['price_per_sqm'] = price_per_sqm

# specs_content = soup.findAll('div', 're__pr-specs-content-item')

# # Find the span containing the posted date
# posted_date_span = soup.find('span', string='Ngày đăng')

# # Extract the value of the posted date
# posted_date = posted_date_span.find_next_sibling('span').text.strip()
# property_details['posted_date'] = posted_date


# # Mapping of titles to corresponding keys in property_details
# title_map = {
#     'Diện tích': 'area',
#     'Mức giá': 'price',
#     'Giấy tờ phjáp lý': 'legal_document',
#     'Nội thất': 'interior',
#     'Số phòng ngủ': 'num_bedrooms',
#     'Số phòng tắm, vệ sinh': 'num_bathrooms',
#     'Số tầng': 'num_floors',
#     'Hướng nhà': 'house_orientation',
#     'Hướng ban công': 'balcony_orientation',
#     'Đường vào': 'entrance',
#     'Mặt tiền': 'frontage'
# }

# # Extract and map property details
# for item in specs_content:
#     title = item.find(class_='re__pr-specs-content-item-title').text.strip()
#     value = item.find(class_='re__pr-specs-content-item-value').text.strip()
#     if title in title_map:
#         property_details[title_map[title]] = value

# print(property_details)

Nhà mặt phố tại đường Nguyễn Đức Cảnh


In [4]:
df_link_data = pd.read_csv('data/link_data.csv')
df_link_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  20000 non-null  int64 
 1   link        20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


In [5]:
LIST_LINK_PRODUCT = df_link_data['link'].values.tolist()
print(len(LIST_LINK_PRODUCT))

20000


In [6]:
LIST_PRODUCT = []

# Define a function to extract property details
def extract_property_details(url):
    property_details = {
        'type_estate': '',
        'district': '',
        'price_per_sqm': '',
        'posted_date': '',
        'area': '',
        'price': '',
        'legal_document': '',
        'interior': '',
        'num_bedrooms': '',
        'num_bathrooms': '',
        'num_floors': '',
        'house_orientation': '',
        'balcony_orientation': '',
        'entrance': '',
        'frontage': ''
    }

    try:
        # Set up Selenium WebDriver
        with webdriver.Chrome() as driver:
            driver.get(url)
            
            # Get the page source
            page_source = driver.page_source

        soup = BeautifulSoup(page_source, "html.parser")

        # Find the breadcrumb element
        breadcrumb = soup.find('div', class_='re__breadcrumb')

        level_4_link = breadcrumb.find('a', {'level': '4'})
        level_4_title = level_4_link.text

        # Use regular expressions to extract the type_estate
        match = re.search(r"(.+?) tại", level_4_title)
        type_estate = match.group(1) if match else ""
        property_details['type_estate'] = type_estate

        level_3_link = breadcrumb.find('a', {'level': '3'})
        property_details['district'] = level_3_link.text

        # Extract price per square meter
        price_per_sqm_span = soup.find('span', class_='ext')
        if price_per_sqm_span:
            property_details['price_per_sqm'] = price_per_sqm_span.text.strip()

        # Extract posted date
        posted_date_span = soup.find('span', string='Ngày đăng')
        property_details['posted_date'] = posted_date_span.find_next_sibling('span').text.strip()

        # Extract property specifications
        specs_content = soup.find_all('div', class_='re__pr-specs-content-item')

        # Mapping of titles to corresponding keys in property_details
        title_map = {
            'Diện tích': 'area',
            'Mức giá': 'price',
            'Pháp lý': 'legal_document',
            'Nội thất': 'interior',
            'Số phòng ngủ': 'num_bedrooms',
            'Số toilet': 'num_bathrooms',
            'Số tầng': 'num_floors',
            'Hướng nhà': 'house_orientation',
            'Hướng ban công': 'balcony_orientation',
            'Đường vào': 'entrance',
            'Mặt tiền': 'frontage'
        }

        # Extract and map property details
        for item in specs_content:
            title = item.find(class_='re__pr-specs-content-item-title').text.strip()
            if title in title_map:
                property_details[title_map[title]] = item.find(class_='re__pr-specs-content-item-value').text.strip()
        
    except Exception as e:
        print(f"Failed to extract data from {url}: {e}")
        raise  # Re-raise the exception to stop the process

    return property_details

def scrape_property_details(urls):
    scraped_properties = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(extract_property_details, url) for url in urls]
        for future in as_completed(futures):
            try:
                result = future.result()
                print(result)  # Print the extracted details
                scraped_properties.append(result)
            except Exception as e:
                print(f"Failed to extract data: {e}")
                break  # Stop further processing to check the issue
    return scraped_properties

LIST_PRODUCT = scrape_property_details(LIST_LINK_PRODUCT)

{'type_estate': 'Nhà mặt phố', 'district': 'Hoàng Mai', 'price_per_sqm': '~158,51 triệu/m²', 'posted_date': '13/05/2024', 'area': '220,8 m²', 'price': '35 tỷ', 'legal_document': 'Sổ đỏ/ Sổ hồng', 'interior': 'Không nội thất', 'num_bedrooms': '3 phòng', 'num_bathrooms': '2 phòng', 'num_floors': '3 tầng', 'house_orientation': 'Tây - Nam', 'balcony_orientation': 'Nam', 'entrance': '10 m', 'frontage': '6,3 m'}
{'type_estate': 'Nhà biệt thự, liền kề', 'district': 'Hà Đông', 'price_per_sqm': '~130 triệu/m²', 'posted_date': '13/05/2024', 'area': '100 m²', 'price': '13 tỷ', 'legal_document': '', 'interior': '', 'num_bedrooms': '', 'num_bathrooms': '', 'num_floors': '', 'house_orientation': '', 'balcony_orientation': '', 'entrance': '', 'frontage': ''}
{'type_estate': 'Nhà biệt thự, liền kề', 'district': 'Hà Đông', 'price_per_sqm': '~173,12 triệu/m²', 'posted_date': '13/05/2024', 'area': '80 m²', 'price': '13,85 tỷ', 'legal_document': 'Sổ đỏ/ Sổ hồng', 'interior': 'Đầy đủ', 'num_bedrooms': '6 p

In [ ]:
print(len(LIST_PRODUCT))

0


In [ ]:
df_link_product = pd.DataFrame(LIST_PRODUCT)
df_link_product.to_csv('data/raw_data.csv')

In [ ]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv("data/raw_data.csv")

# Drop the 'link' and 'describe' columns
df.drop(['link', 'describe'], axis=1, inplace=True)

# Save the modified DataFrame back to a CSV file
df.to_csv("data/raw_data.csv", index=False)

In [ ]:
df = pd.read_csv('data/raw_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    4000 non-null   int64  
 1   estate_type   3210 non-null   object 
 2   province      3210 non-null   object 
 3   district      3206 non-null   object 
 4   ward          3206 non-null   object 
 5   price         3824 non-null   object 
 6   square        3770 non-null   object 
 7   post_date     4000 non-null   object 
 8   numb_bedroom  3076 non-null   float64
 9   numb_toilet   2864 non-null   float64
 10  numb_floor    3155 non-null   float64
dtypes: float64(3), int64(1), object(7)
memory usage: 343.9+ KB
